In [1]:
import os
import pandas as pd

base_dir = 'dataset/flowers-recognition_configed'
input_data_dir = {'test':None, 'train':None, 'validation':None}

totalcounter = 0
dfs = {'train':None, 'validation':None, 'test':None}
for i in input_data_dir:
    print(i)
    classes_names = os.listdir(os.path.join(base_dir, i))
    input_data_dir[i] = os.path.join(base_dir, i)
    classes = {}
    for s in classes_names:
        classes[s] = len(os.listdir(os.path.join(base_dir, i, s)))
        
    inputdata = {'flower':None, 'count':None}
    inputdata['flower'] = [f for f, c in classes.items()]
    inputdata['count'] = [c for f, c in classes.items()]
    
    dfs[i] = pd.DataFrame(inputdata)
    totalcounter += dfs[i]['count'].sum()
    print('\t', 'image count mean:', int(dfs[i]['count'].mean()))
    print('\t', 'image count :', int(dfs[i]['count'].sum()))
print('-'.join(['' for i in range(0, 16)]))
print('total image count:', totalcounter)    

class_count = len(classes)

test
	 image count mean: 183
	 image count : 915
train
	 image count mean: 368
	 image count : 1840
validation
	 image count mean: 181
	 image count : 905
---------------
total image count: 3660


# hyper parameters

In [2]:
from keras import optimizers
import math

h_params = {}

# image
h_params['image_size'] = 150 # defult 150
h_params['batch_size'] = 20 # defult 20

# fit
h_params['epochs'] = 40
h_params['steps_per_epoch'] = math.ceil(dfs['train']['count'].sum()/h_params['batch_size']) # defult 100
h_params['validation_steps'] = math.ceil(dfs['validation']['count'].sum()/h_params['batch_size']) # defult 50

# compile
h_params['learning_rate'] = 1e-4 # defult 2e-5
h_params['optimizer'] = optimizers.RMSprop(lr=h_params['learning_rate'])
h_params['metrics'] = ['acc']
h_params['fine_tuning_startlayer'] = 'block5_conv1' # if None means no fine tuning is used.

Using TensorFlow backend.


In [3]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                include_top=False,
                input_shape=(h_params['image_size'],
                             h_params['image_size'],
                             3))

conv_base.summary()
print('layer count', len(conv_base.layers))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [4]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(class_count, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 16,813,381
Trainable params: 16,813,381
Non-trainable params: 0
_________________________________________________________________


In [5]:
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))

if h_params['fine_tuning_startlayer'] == None:   
    conv_base.trainable = False
    for i in conv_base.layers:
        i.trainable = False
else:
    conv_base.trainable = True
    set_trainable = False
    for layer in conv_base.layers:
        if layer.name == h_params['fine_tuning_startlayer']:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
print('This is the number of trainable weights ''after freezing the conv base:', len(model.trainable_weights))    

model.compile(loss='categorical_crossentropy',
                optimizer=h_params['optimizer'],
                metrics=h_params['metrics'])

In [6]:
for i in conv_base.layers:
    print(i.name, '\t',i.trainable)

input_1 	 False
block1_conv1 	 False
block1_conv2 	 False
block1_pool 	 False
block2_conv1 	 False
block2_conv2 	 False
block2_pool 	 False
block3_conv1 	 False
block3_conv2 	 False
block3_conv3 	 False
block3_pool 	 False
block4_conv1 	 False
block4_conv2 	 False
block4_conv3 	 False
block4_pool 	 False
block5_conv1 	 True
block5_conv2 	 True
block5_conv3 	 True
block5_pool 	 True


# preprocessing

In [7]:
train_dir = input_data_dir['train']
validation_dir = input_data_dir['validation']

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                #rotation_range=40,
                                #width_shift_range=0.2,
                                #height_shift_range=0.2,
                                #shear_range=0.2,
                                #zoom_range=0.2,
                                #horizontal_flip=True,
                                  )
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(h_params['image_size'], h_params['image_size']),
                                                    batch_size=h_params['batch_size'],
                                                    class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(h_params['image_size'], h_params['image_size']),
                                                        batch_size=h_params['batch_size'],
                                                        class_mode='categorical')

Found 1840 images belonging to 5 classes.
Found 905 images belonging to 5 classes.


In [8]:
#### from keras import models
from keras import layers
from keras import optimizers
import time
import tensorflow as tf

startt = time.time()

with tf.device('/GPU:0'):
    history = model.fit_generator(train_generator,
                              steps_per_epoch=h_params['steps_per_epoch'],
                              epochs=h_params['epochs'],
                              validation_data=validation_generator,
                              validation_steps=h_params['validation_steps'])

# save model
import helper as hlp
model_h = {}
model_h['history'] = history
model_h['train_duration'] = round(time.time() - startt)
model_h['hyper_parameters'] = h_params
model_h['hyper_parameters']['optimizer'] = str(h_params['optimizer'])
filename = str(round(time.time()))
filename = '_'.join([filename[:5], filename[5:6], filename[6:7], filename[7:8], filename[8:], 'pretrained_vgg16_embeded'])
hlp.save(model_h, 'models/'+filename)

Epoch 1/40
92/92 [==============================] - 50s 548ms/step - loss: 0.8711 - acc: 0.6712 - val_loss: 0.5715 - val_acc: 0.7945
Epoch 2/40
92/92 [==============================] - 45s 493ms/step - loss: 0.4273 - acc: 0.8478 - val_loss: 0.4383 - val_acc: 0.8409
Epoch 3/40
92/92 [==============================] - 46s 495ms/step - loss: 0.2166 - acc: 0.9261 - val_loss: 0.3648 - val_acc: 0.8707
Epoch 4/40
92/92 [==============================] - 45s 492ms/step - loss: 0.1457 - acc: 0.9582 - val_loss: 0.3680 - val_acc: 0.8818
Epoch 5/40
92/92 [==============================] - 45s 491ms/step - loss: 0.0700 - acc: 0.9815 - val_loss: 0.4388 - val_acc: 0.8840
Epoch 6/40
92/92 [==============================] - 45s 491ms/step - loss: 0.0653 - acc: 0.9826 - val_loss: 0.4065 - val_acc: 0.8928
Epoch 7/40
92/92 [==============================] - 45s 491ms/step - loss: 0.0486 - acc: 0.9864 - val_loss: 0.5087 - val_acc: 0.8961
Epoch 8/40
92/92 [==============================] - 45s 492ms/step - 

In [ ]:
# show result with plot
print('model file name is:', filename)
hlp.show_model_plot(model_h);

# final learning

In [8]:
#### from keras import models
from keras import layers
from keras import optimizers
import time
import tensorflow as tf

startt = time.time()

with tf.device('/GPU:0'):
    history = model.fit_generator(train_generator,
                              steps_per_epoch=h_params['steps_per_epoch'],
                              epochs=9,
                              validation_data=validation_generator,
                              validation_steps=h_params['validation_steps'])

# save model
import helper as hlp
model_h = {}
model_h['history'] = history
model_h['train_duration'] = round(time.time() - startt)
model_h['hyper_parameters'] = h_params
model_h['hyper_parameters']['optimizer'] = str(h_params['optimizer'])
filename = str(round(time.time()))
filename = '_'.join([filename[:5], filename[5:6], filename[6:7], filename[7:8], filename[8:], 'pretrained_vgg16_embeded_bestmodel'])
hlp.save(model_h, 'models/'+filename)

Epoch 1/9
92/92 [==============================] - 51s 550ms/step - loss: 0.8692 - acc: 0.6620 - val_loss: 0.6797 - val_acc: 0.7392
Epoch 2/9
92/92 [==============================] - 46s 497ms/step - loss: 0.4188 - acc: 0.8435 - val_loss: 0.4361 - val_acc: 0.8365
Epoch 3/9
92/92 [==============================] - 46s 496ms/step - loss: 0.2353 - acc: 0.9223 - val_loss: 0.3646 - val_acc: 0.8807
Epoch 4/9
92/92 [==============================] - 46s 496ms/step - loss: 0.1274 - acc: 0.9592 - val_loss: 0.4256 - val_acc: 0.8950
Epoch 5/9
92/92 [==============================] - 46s 496ms/step - loss: 0.0933 - acc: 0.9728 - val_loss: 0.3624 - val_acc: 0.8851
Epoch 6/9
92/92 [==============================] - 46s 496ms/step - loss: 0.0474 - acc: 0.9870 - val_loss: 0.3386 - val_acc: 0.8829
Epoch 7/9
92/92 [==============================] - 46s 495ms/step - loss: 0.0596 - acc: 0.9842 - val_loss: 0.4592 - val_acc: 0.8917
Epoch 8/9
92/92 [==============================] - 45s 494ms/step - loss: 0.